## Code to fetch Azure Pricing data from Azure API and save the dataset into ADSL in CSV format 

In [ ]:
import json
import requests
import csv
import pathlib
import pandas as pd
from io import StringIO
from io import BytesIO

In [70]:
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings

In [71]:
url="https://prices.azure.com/api/retail/prices?$filter=serviceName eq 'Virtual Machines'"
#AzurePricing=requests.get(url)


In [72]:
def join(a,b):
    k=pd.merge(left=a,left_index=True, right=b,right_index=True, how='inner')
    return k

In [156]:
def Price_Data_Extraction(url):
    PriceData=requests.get(url)
    PriceDataTxt=PriceData.text
    PriceData_Json=json.loads(PriceDataTxt)
    df_main=pd.DataFrame.from_dict(PriceData_Json)
    url=df_main['NextPageLink'].unique()[0]
    itemdata=[]
    for i in df_main['Items']:
        itemdata.append(i)
    df_items=pd.DataFrame.from_dict(itemdata)
    
    PriceData=join(df_main,df_items)
    PriceData=PriceData.drop(['Items','NextPageLink'],axis=1)
    
    #return df_items, df_main
    return PriceData


In [74]:
df=Price_Data_Extraction(url)


### Code to save dataframe into ADSL in CSV file format

In [89]:
def Upload_CSV_File(storage_account_name,storageaccountkey,storagecontainer,cont_directory,DataframeToCopy):
    try:
        storageaccountconnection = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format("https", storage_account_name), credential=storageaccountkey)
        
        ContainerConnection = storageaccountconnection.get_file_system_client(file_system=storagecontainer)
        directory_client = ContainerConnection.get_directory_client(cont_directory)
        file_client = directory_client.create_file("AzureVMPricing.csv")
        
        text_stream = StringIO()
        DataframeToCopy.to_csv(text_stream)
        file_client.upload_data(text_stream.getvalue(),overwrite=True)
        file_client.flush_data(len(text_stream.getvalue()))
    except Exception as e:
      print(e)

## Download csv file from ADSL and convert the data into Parquet format and save it back into ADSL

In [154]:
def CSV_To_Parquet_File(storage_account_name,storageaccountkey,storagecontainer,cont_directory,filenameCSV):
    try:
        stat=0
        if len(filenameCSV) and filenameCSV.find(".")>0:
            fname=filenameCSV.split(".")
        else:
            stat=1
        
        if stat==0:
            if len(filenameCSV) and fname[1].casefold()=="csv" and len(fname[0]):    
            
                storageaccountconnection = DataLakeServiceClient(account_url="{}://{}.dfs.core.windows.net".format("https", storage_account_name), credential=storageaccountkey)
        
                ContainerConnection = storageaccountconnection.get_file_system_client(file_system=storagecontainer)
                directory_client = ContainerConnection.get_directory_client(cont_directory)
        
                #file_client_read = directory_client.get_file_client("AzureVMPricing.csv")
                file_client_read = directory_client.get_file_client(filenameCSV)
        
                DataFile=file_client_read.download_file()
                DataFile_bytes=download.readall()
        
                data = BytesIO(downloaded_bytes)
                df_fromcsv=pd.read_csv(data)
                df_p=pd.DataFrame(df_fromcsv).to_parquet()
                file_client_parquet = directory_client.create_file("AzureVMPricing.parquet")
                file_client_parquet.append_data(data=df_p, offset=0, length=len(df_p))
                file_client_parquet.flush_data(len(df_p))
            else:
                stat=1
                
        
        if stat==1:
            return "Success"
        else:
            return "Invalid Filename"
              
                                        
        #return df_fromcsv
    except Exception as e:
      print(e)